Google Colaboratory 환경에서 실행했습니다. (T4 고용량 RAM)

# 수정된 diffractsim 설치

In [ ]:
!pip install git+https://github.com/hjw4228/diffractsim

In [ ]:
import gc

import diffractsim
diffractsim.set_backend("CUDA")

from diffractsim import PolychromaticField, Lens, ApertureFromImage, cf, nm, mm, cm

def render_image(f, z, mask):
  F = PolychromaticField(
    spectrum = 4 * cf.illuminant_d65,
    extent_x = 15.*mm, extent_y = 15.*mm,
    Nx=1500, Ny=1500
  )
  if mask:
    F.add(ApertureFromImage("./mask.jpg", image_size=(5. *mm, 5.* mm), simulation=F))
  else:
    F.add(ApertureFromImage("./circle.jpg", image_size=(5. *mm, 5. *mm), simulation=F))

  F.add(Lens(f=f*cm))
  F.propagate(z=z * cm)

  rgb = F.get_colors()

  if mask:
    F.plot_colors(rgb, xlim=[-6*mm, 6*mm], ylim=[-6*mm, 6*mm], save_plot=True, filename=f'{f}_{z}_masked.jpg', hide_axis=True, text='')
    print(f'\nSuccessfully rendered masked image of f={f}cm, z={F.z * 100}cm')
  else:
    F.plot_colors(rgb, xlim=[-6*mm, 6*mm], ylim=[-6*mm, 6*mm], save_plot=True, filename=f'{f}_{z}_unmasked.jpg', hide_axis=True, text='')
    print(f'\nSuccessfully rendered unmasked image of f={f}cm, z={F.z * 100}cm')\

  return

In [ ]:
def render_all(start, end):
  for f in range(start, end+1):
    z = f-10
    for _ in range(1, 20):
      render_image(f, z, False)
      z += 1
    print(f'Collected garbage: {gc.collect()}')
    z = f-5

In [ ]:
gc.collect()
render_all(75, 80)